# **Metagenomics Pipeline**

### Metagenomic amplicon sequencing -> single or paired end fatsq 

Data: https://drive.google.com/drive/u/2/folders/1EbrItGAVVB75YAgNtEwpO-ztFeVxJqrV 

### Pipeline :

Raw reads --(QIIME)--> ASV --(Picrust2)--> Bact

Raw reads --(QIIME)--> ASV --(FUNGuild)--> Fungi

![## QIIME](firstsec.png)

## Importing files for paired-end analysis


In [ ]:
qiime tools import \
  --type 'SampleData[PairedEndSequencesWithQuality]' \
  --input-path paired-files \
  --input-format CasavaOneEightSingleLanePerSampleDirFmt \
  --output-path paired-end-sequences.qza

SyntaxError: invalid syntax (1707935972.py, line 1)

## Demultiplexed samples

In [ ]:
qiime demux summarize \
--i-data paired-end-sequences.qza \
--o-visualization demux.qzv

## Demultiplexing



In [ ]:
qiime demux emp-paired \
  --m-barcodes-file LMS-16S-metadata.tsv \
  --m-barcodes-column sample-id \
  --p-rev-comp-mapping-barcodes \
  --i-seqs paired-end-sequences.qza \
  --o-per-sample-sequences demux-full.qza \
  --o-error-correction-details demux-details.qza

## Trimming

300 bp reads

The sequences for the commonly used bacterial primers **BAkt_341F** and **BAkt_805R** are:

- **BAkt_341F (Forward Primer)**: `CCTACGGGNGGCWGCAG`
- **BAkt_805R (Reverse Primer)**: `GACTACHVGGGTATCTAATCC`

These primers are used to amplify the **V3-V4 region** of the **16S rRNA gene** in bacteria. This region is often targeted in microbial community studies because it provides enough variability to distinguish between different bacterial taxa while being conserved enough to allow universal primer binding.



| Characteristic       | Primers                                  | Adapters                                   |
|----------------------|-------------------------------------------|--------------------------------------------|
| **Function**         | Initiate DNA synthesis during PCR.       | Enable binding of DNA fragments to the sequencing platform. |
| **Location**         | Bind to the **5' end** of target regions during PCR. | Added to both **3' and 5' ends** of DNA fragments during library preparation. |
| **Role**             | Define the region of DNA to be amplified. | Facilitate attachment of DNA to sequencing machinery. |
| **Removal During Analysis** | Usually removed during quality control to ensure only the target sequence is analyzed. | Often trimmed to remove unwanted sequences before downstream analysis. |



### Primer

--pcores 4 is for the MacBookAir 

In [ ]:
qiime cutadapt trim-paired \
--i-demultiplexed-sequences paired-end-sequences.qza \
--p-cores 4 \
--p-front-f "CCTACGGGNGGCWGCAG" \
--p-front-r "GACTACHVGGGTATCTAATCC" \
--o-trimmed-sequences front-trimmed-seqs.qza

In [ ]:
qiime demux summarize \
--i-data front-trimmed-seqs.qza \
--o-visualization trimmed-seqs.qzv

## Denoising

In [ ]:
qiime dada2 denoise-paired \
    --i-demultiplexed-seqs demux-paired-end.qza \
    --p-trunc-len-f 250 \
    --p-trunc-len-r 170 \
    --p-trim-left-f 6   \
    --p-trim-left-r 6   \
    --o-representative-sequences rep-seqs.qza \
    --o-table freq-feature-table.qza \
    --o-denoising-stats denoising-stats.qza